<h1>Polygon Time Series Model</h1>
A time series model using the Polygon.io API.

In [ ]:
#import necessary libraries
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt
import seaborn as sns
import requests
import warnings;
warnings.simplefilter('ignore')
today = date.today()

Below, input your API key, the stock ticker of interest, the start date, and the amount of periods to forecast into the future.

In [ ]:
#input all parameters
api_key = "XXXXXX"
stock_ticker = "XXXX"
start_date = "YYYY-MM-DD"
end_date = today.strftime("%Y-%m-%d")
#input future length
periods = 60

In [ ]:
#pulls data from Polygon
get_request = "https://api.polygon.io/v2/aggs/ticker/" + stock_ticker + "/range/1/day/" + start_date + "/" + end_date + "?adjusted=true&sort=asc&limit=50000&apiKey=" + api_key
request = requests.get(get_request)

In [ ]:
#create dataframe
df1 = pd.read_json(request.content)
df = pd.DataFrame(df1.results.values.tolist())
df.columns = ['volume', 'volume_weighted_avg_price', 'open_price', 'close_price', 'high', 'low', 'time', 'transactions']
df.head()

In [ ]:
#convert Unix time to Y/M/D
df['date'] = df['time'].apply(lambda x: date.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
df.drop(columns=['time'], inplace=True)
df.head()

<h1>Prophet Time Series:</h1>

In [ ]:
#import prophet, create dataframe, and fit model
from prophet import Prophet
close_price_df = df[['date','close_price']]
close_price_df.columns = ['ds', 'y']
close_price_df['ds'] = pd.DatetimeIndex(close_price_df['ds'])
model = Prophet(interval_width = .95, daily_seasonality=True)
model.fit(close_price_df)


In [ ]:
#create future
future = model.make_future_dataframe(periods)
#predict with prophet model
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
fig1 = model.plot(forecast)

Above, we have a plot of the stock price data points, with the line of the Time Series plotted throughout. This will be different for each dataset and model.

In [ ]:
fig2 = model.plot_components(forecast)

Above, the three plots show the component importance. This will be different for each model.